# 7 Algorithms Regression

#### We use 7 algorithms regression

* ExtraTrees Regressor
* AdaBoost Regressor
* Bayesian Ridge
* Linear Regression 
* Random Forest Regressor
* XGB Regressor
* LGBM Regressor





<img src="https://analyticsindiamag.com/wp-content/uploads/2018/01/bigstock-175575391.jpg" width="800px">



**References Link**


##### [Here](https://scikit-learn.org/stable/modules/svm.html#regression)


## Data Description

### File descriptions

* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.



### Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category


### Dataset Link



#### [Here](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data)

In [ ]:
pip install xlrd

In [ ]:
pip install autoviz

In [ ]:
# Import Lab
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from scipy import optimize, stats   
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from keras.utils import np_utils

In [ ]:
df_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()
target='item_price'
df = AV.AutoViz(filename="",sep=',', depVar=target, dfte=df_train, header=0, verbose=1, 
                 lowess=False, chart_format='svg', max_rows_analyzed=150000, max_cols_analyzed=30)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_train)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_shops)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_items)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_test)

In [ ]:
df_train.drop(['date_block_num','item_price'], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], dayfirst=True)
df_train['date'] = df_train['date'].apply(lambda x: x.strftime('%Y-%m'))
df_train.head()

In [ ]:
df = df_train.groupby(['date','shop_id','item_id']).sum()
df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head().T

In [ ]:
df_test = pd.merge(df_test, df, on=['shop_id','item_id'], how='left')
df_test.drop(['ID', '2013-01'], axis=1, inplace=True)
df_test = df_test.fillna(0)
df_test.head().T

In [ ]:
# split into train and test sets
Y_train = df['2015-10'].values
X_train = df.drop(['2015-10'], axis = 1)
X_test = df_test

print(X_train.shape, Y_train.shape)
print(X_test.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( X_train, Y_train, test_size=0.20, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
Y_test

In [ ]:
%time
ETR = ExtraTreesRegressor(n_estimators=100, random_state=0)
ETR.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, ETR.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, ETR.predict(X_test)))
print('Test set score:', ETR.score(X_train,Y_train))

In [ ]:
%time
ADB= AdaBoostRegressor(random_state=0, n_estimators=100)
ADB.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, ADB.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, ADB.predict(X_test)))
print('Test set score:', ADB.score(X_train,Y_train))

In [ ]:
%time
BYNR = linear_model.BayesianRidge()
BYNR.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, BYNR.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, BYNR.predict(X_test)))
print('Test set score:', BYNR.score(X_train,Y_train))

In [ ]:
%time
LR = LinearRegression()
LR.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, LR.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, LR.predict(X_test)))
print('Test set score:', LR.score(X_train,Y_train))

In [ ]:
%time
RFR = RandomForestRegressor(n_estimators = 100)
RFR.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, RFR.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, RFR.predict(X_test)))
print('Test set score:', RFR.score(X_train,Y_train))

In [ ]:
%time
XGB = XGBRegressor(max_depth=16,n_estimators=200,seed=1)
XGB.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, XGB.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, XGB.predict(X_test)))
print('Test set score:', XGB.score(X_train,Y_train))

In [ ]:
%time
LGBM = LGBMRegressor(max_depth=16,n_estimators=200,seed=1)
LGBM.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, LGBM.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, LGBM.predict(X_test)))
print('Test set score:', LGBM.score(X_train,Y_train))

In [ ]:
prediction = XGB.predict(X_test)

In [ ]:
prediction[5]

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
score_1 = 0.76 
score_2 = 0.87
score_3 = 0.43
score_4 = 0.43
score_5 = 0.88
score_6 = 0.99
score_7 = 0.70 

results = pd.DataFrame([["ExtraTreesRegressor",score_1],["AdaBoostRegressor",score_2],[" BayesianRidge",score_3],
                       ["LinearRegression",score_4],["RandomForestRegressor",score_5],
                       ["XGBRegressor",score_6],["RLGBMRegressor",score_7]],
                        columns = ["Models","Score"]).sort_values(by='Score',ascending=False)


results.style.background_gradient(cmap='Blues')